In [1]:
import os
import numpy as np

from astropy.io import fits
from matplotlib import pyplot as plt
import numpy as np
import lightkurve as lk
from astroquery.simbad import Simbad
import pandas as pd 
from numpy import genfromtxt

# #tf
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

from numba import jit, njit, vectorize

In [2]:
labeled_data = pd.read_csv('/Users/michaelsong/Documents/GitHub/starzam/Labels/labeled_stars.csv',delimiter=',')
print(labeled_data)
unlabeled_data = pd.read_csv('/Users/michaelsong/Documents/GitHub/starzam/Labels/unlabeled_stars.csv',delimiter=',')
print(unlabeled_data)

        Kepler_ID          2MASS ID  Teff  Log_G  Metallicity (solar=0.0)  \
0          757076  19240929+3635532  5174  3.601                   -0.083   
1          757099  19241033+3635377  5589  3.817                   -0.208   
2          757137  19241341+3633358  4879  2.578                   -0.079   
3          757218  19241934+3635394  4555  2.281                   -0.120   
4          757231  19241988+3634264  4825  2.601                   -0.082   
...           ...               ...   ...    ...                      ...   
191444   12984214  19201777+5226518  5267  3.818                    0.128   
191445   12984288  19202700+5224561  6251  4.297                   -0.177   
191446   12984307  19202959+5228286  5553  4.221                   -0.172   
191447   12984404  19204135+5225258  4604  2.409                    0.394   
191448   12984422  19204420+5224157  5078  4.569                    0.256   

        E(B-V)    A_V  Radius (solar=1.0)  Photometry_Qual  Astrophysics_Qu

In [3]:
labeled_data_rows = labeled_data.T
print(labeled_data_rows[0])

Kepler_ID                            757076
2MASS ID                   19240929+3635532
Teff                                   5174
Log_G                                 3.601
Metallicity (solar=0.0)              -0.083
E(B-V)                                0.093
A_V                                   0.289
Radius (solar=1.0)                    3.029
Photometry_Qual                           7
Astrophysics_Qual                         6
Name: 0, dtype: object


In [19]:
def wrap(fn):
    try:
        return fn()
    except:
        pass

@jit
def prepsr(i,sr,j):
    lightcurve = sr[j].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
    periodogram = lightcurve.to_periodogram(frequency = [x for x in np.arange(1,48,0.01)])
    periodogram = periodogram.bin(binsize=10, method='mean')
    freq = np.asarray([periodogram.frequency,periodogram.power*100000])
    # pd.DataFrame(freq[1]).to_csv("/content/drive/MyDrive/Research - Yihong Song and Rohit Prasanna/Yihong/Data/"+str(labeled_data["Kepler_ID"][i])+"_"+str(j)+".csv")
    return np.append(np.append(labeled_data_rows[i],[j]),freq[1])

@jit
def getlc(i):
    lc_data = []
    sr = lk.search_lightcurve('KIC '+str(labeled_data["Kepler_ID"][i]), author='Kepler',exptime=1800)
    # return prepsr(i,sr,1)
    for j in range(len(sr)):
        # x = wrap(prepsr(i,sr,j))
        # if x is not None:
        lc_data.append(prepsr(i,sr,j))
        # y = wrap(external_function_raises)
        # if y is not None:
        #     lc_data.append(y)
    return lc_data

def getdata(start):
    # for k in range(start,int(191449/100)+1,1):
    for k in range(start,start+5,1):
        lc_data = []
        for i in range(k*100,(k+1)*100,1):
            try:
                lc_data = lc_data+getlc(i)
            except:
                continue
            print("\r"+str(i),end="")
        # print(lc_data)
        pd.DataFrame(lc_data).to_csv("/Users/michaelsong/Documents/GitHub/starzam/Data_v2/"+str(k)+".csv")
            
getdata(45)

/var/folders/7x/d_9j9fy937g6_216hzt6gxm00000gn/T/ipykernel_43310/2802009912.py:16: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "getlc" failed type inference due to: Untyped global name 'labeled_data': Cannot determine Numba type of <class 'pandas.core.frame.DataFrame'>

File "../../../../../../var/folders/7x/d_9j9fy937g6_216hzt6gxm00000gn/T/ipykernel_43310/2802009912.py", line 19:
<source missing, REPL/exec in use?>

  @jit
/var/folders/7x/d_9j9fy937g6_216hzt6gxm00000gn/T/ipykernel_43310/2802009912.py:16: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "getlc" failed type inference due to: Untyped global name 'labeled_data': Cannot determine Numba type of <class 'pandas.core.frame.DataFrame'>

File "../../../../../../var/folders/7x/d_9j9fy937g6_216hzt6gxm00000gn/T/ipykernel_43310/2802009912.py", line 19:
<source missing, REPL/exec in use?>

  @jit
/Users/michaelsong/miniforge3

4906

In [14]:

getlc(1)

/var/folders/7x/d_9j9fy937g6_216hzt6gxm00000gn/T/ipykernel_43310/901176795.py:7: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "prepsr" failed type inference due to: Untyped global name 'labeled_data_rows': Cannot determine Numba type of <class 'pandas.core.frame.DataFrame'>

File "../../../../../../var/folders/7x/d_9j9fy937g6_216hzt6gxm00000gn/T/ipykernel_43310/901176795.py", line 14:
<source missing, REPL/exec in use?>

  @jit
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "prepsr" was compiled in object mode without forceobj=True.

File "../../../../../../var/folders/7x/d_9j9fy937g6_216hzt6gxm00000gn/T/ipykernel_43310/901176795.py", line 7:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaWarning(warn_msg,
/Users/michaelsong/miniforge3/lib/python3.9/site-packages/numba/core/object_mode_passes.py:161: NumbaDeprecationWarning: 
Fall-back from th

[array([757099, '19241033+3635377', 5589, 3.817, -0.208, 0.145, 0.448,
        2.288, 7, 6, 0, 11.858734396839399, 17.713708490178785,
        15.48073132933265, 10.652535011729798, 21.760184106955137,
        19.25680432907391, 22.28958245787651, 18.568109487295008,
        29.875434697007602, 27.549806372066612, 24.433422596194823,
        24.72649738983311, 30.475127144387077, 33.39450580234083,
        34.11046333909232, 68.67952317608766, 142.11466410482902,
        788.0851445513789, 195.81395110489456, 70.69344654363893,
        43.25304830962287, 29.746609053732442, 27.12210407159139,
        17.924172516427152, 18.33054518271358, 13.699582445625811,
        16.054950652852497, 12.145595949500738, 15.34501754804792,
        16.381411481366158, 12.159936776769221, 13.150305733391175,
        9.869314685506923, 12.610108595125437, 14.047112189654476,
        12.726768525781463, 11.254349977167818, 12.029828051052332,
        13.851900748759887, 18.12078484719626, 13.0644469583974

In [17]:
# jitted_function = jit()(getlc)

In [ ]:
for k in range(38,int(191449/100)+1,1):
  lc_data = []
  for i in range(k*100,(k+1)*100,1):
    try:
      sr = lk.search_lightcurve('KIC '+str(labeled_data["Kepler_ID"][i]), author='Kepler',exptime=1800)
      for j in range(len(sr)):
        lightcurve = sr[j].download().remove_outliers(sigma=20, sigma_upper=4).normalize()
        periodogram = lightcurve.to_periodogram(frequency = [x for x in np.arange(1,48,0.01)])
        periodogram = periodogram.bin(binsize=10, method='mean')
        freq = np.asarray([periodogram.frequency,periodogram.power*100000])
        # pd.DataFrame(freq[1]).to_csv("/content/drive/MyDrive/Research - Yihong Song and Rohit Prasanna/Yihong/Data/"+str(labeled_data["Kepler_ID"][i])+"_"+str(j)+".csv")
        lc_data.append(np.append(np.append(labeled_data_rows[i],[j]),freq[1]))
    except:
      continue
    print("\r"+str(i)+" "+str(len(lc_data[i%100])),end="")
  pd.DataFrame(lc_data).to_csv("/Users/michaelsong/Documents/GitHub/starzam/Data_v2/"+str(k)+".csv")


Kepler_ID                            757076
2MASS ID                   19240929+3635532
Teff                                   5174
Log_G                                 3.601
Metallicity (solar=0.0)              -0.083
E(B-V)                                0.093
A_V                                   0.289
Radius (solar=1.0)                    3.029
Photometry_Qual                           7
Astrophysics_Qual                         6
Name: 0, dtype: object
3906 481